In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable, grad
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
torch.manual_seed(0)

In [21]:
import torch
import torch
from torch.utils.data import Dataset, DataLoader, Sampler

class BernoulliSampler(Sampler):
    def __init__(self, data_source, p=0.5):
        """ 伯努利采样器初始化
        参数:
        data_source (Dataset): 数据集，用于确定采样总数
        p (float): 每个样本被选中的概率
        """
        self.data_source = data_source
        self.p = p

    def __iter__(self):
        n = len(self.data_source)
        # 生成随机决定，1 表示选择该索引，0 表示不选择
        selection = torch.bernoulli(torch.full((n,), self.p))
        # 选择被选中的索引
        return iter([i for i, flag in enumerate(selection) if flag == 1])

    def __len__(self):
        # 期望长度是总长度乘以选择概率 p
        return int(len(self.data_source) * self.p)

In [23]:
class SimpleDataset(Dataset):
    def __init__(self):
        self.data = torch.arange(1, 101)  

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# 创建数据集
dataset = SimpleDataset()
# 设置抽样参数
p = 0.8
# 创建 BernoulliSampler 实例
sampler = BernoulliSampler(dataset, p=p)
# 创建数据加载器
loader = DataLoader(dataset, sampler=sampler)

# 初始化样本计数数组
sample_counts = np.zeros(len(dataset)) 

# 开始迭代加载器中的数据
for batch in loader:
    # 获取当前批次中的索引
    idx = batch[0].item()
    # 计数数组中对应索引位置加1
    sample_counts[idx-1] += 1

# 输出样本计数和重复索引
print("Sample counts:", sample_counts)
print("Sample indices with duplicates:", np.where(sample_counts > 1)[0])

Sample counts: [1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 0. 0. 0.]
Sample indices with duplicates: []


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader, Sampler
import numpy as np

class BatchBernoulliSampler(Sampler):

    """
        实现有放回的伯努利抽样
    """

    def __init__(self, data_source, batch_size = 10, p=0.7):
        self.data_source = data_source
        self.batch_size = batch_size
        self.p = p  

    def __iter__(self):
        n = len(self.data_source)
        
        while True:
            # 首先随机选择batch_size个样本（有放回）
            batch_indices = torch.randint(0, n, (self.batch_size,)).tolist()
            # 对这些样本进行伯努利抽样
            final_batch = [idx for idx in batch_indices if torch.rand(1).item() < self.p]
            if len(final_batch) > 0:
                yield final_batch

    def __len__(self):
        return int(len(self.data_source) * self.p / self.batch_size)

In [19]:
dataset = SimpleDataset()
batch_size = len(dataset)
p = 0.8  
sampler = BatchBernoulliSampler(dataset, batch_size=batch_size, p=p)

loader = DataLoader(dataset, batch_sampler=sampler)

cnt = 0

sample_counts = np.zeros(len(dataset)) 
cnt = 0
for batch in loader:
    print(batch, len(batch))  
    for idx in batch:
        sample_counts[idx-1] += 1  
    cnt += 1
    if cnt == 3: break

print("Sample counts:", sample_counts)
print("Sample indices with duplicates:", np.where(sample_counts > 1)[0])

tensor([ 16,  38,  90,  23,  42,   1,  20,  28,  61,  36,  85,  29,  58,  66,
         96,  58, 100,  91,  31,  52,  14,  61,  12,  38,   5,  65,  74,  55,
         59,  47,  33,   9,  68,  29,  52,  43,  76,  54,  99,  36,  73,  39,
          9,  55,  70,  85,  11,  20,  74,  49,  81,  80,  13,  87,  69,  82,
         57,  43,  59,  63,  60,  80,  54,  37,   1,  13,  19,  15,  85,  63,
         89,  17,  79,   8,  85,  44,  95,  24,  36]) 79
tensor([ 68,  71,  52,  23,  39,  14,  52,  68,  12,   4,  12,   7,  23,  55,
         16,  87,  62,  72,  34,  50,  33,  25,  99,   1,  46,  91,  61,  26,
         53,   7,  56,  58,  29,  11, 100,  13,  12,  35,  58,  18,  65,  17,
         73,  76,  90,  28,  95,  37,  61,  92,  91,  78,  38,  66,  35,  46,
         78,  55,   3,  99,   3,  70,  88,  88,  73,  67,  81,  78,  54,  56,
         34,  21,  26,  62,  41,  99,  39,  14,  31,  68]) 80
tensor([90, 54, 36, 87, 15, 84, 68,  1, 31, 95, 17, 65, 90, 48, 20, 15, 30, 79,
        35, 80, 37, 2

In [12]:
transform = transforms.Compose(
	[transforms.ToTensor(),
	transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
										download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
											sampler=BernoulliSampler(trainset, p=0.8), num_workers=2)

gradloader = torch.utils.data.DataLoader(trainset, batch_size=4,
											shuffle=False, num_workers=2) #to get the gradient for each epoch

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
										download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
										shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
			'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
#print(' '.join('%5s' % classes[labels[j]] for j in range(4)))
plt.show()

print("原始数据集的长度：", len(trainset))

print(len(trainloader))

Files already downloaded and verified
Files already downloaded and verified
原始数据集的长度： 50000
6250


In [13]:
class Net(nn.Module):
	def __init__(self):
		super(Net, self).__init__()
		self.conv1 = nn.Conv2d(3, 6, 5)
		self.pool = nn.MaxPool2d(2, 2)
		self.conv2 = nn.Conv2d(6, 16, 5)
		self.fc1 = nn.Linear(16 * 5 * 5, 120)
		self.fc2 = nn.Linear(120, 84)
		self.fc3 = nn.Linear(84, 10)

	def forward(self, x):
		x = self.pool(F.relu(self.conv1(x)))
		x = self.pool(F.relu(self.conv2(x)))
		x = x.view(-1, 16 * 5 * 5)
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		x = self.fc3(x)
		return x

In [14]:
n_epoch = 20
print(len(trainloader))
net = Net()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
start = time.time()
n_samples = len(trainloader)
grad_norm_epoch = [0 for i in range(n_epoch)]
full_loss_epoch = [0 for i in range(n_epoch)]

for epoch in range(n_epoch):  # loop over the dataset multiple times
	#Compute full gradient and full loss function
	running_loss = 0.0
	net.zero_grad()
	for i, data in enumerate(gradloader, 0):
		inputs, labels = data
		inputs, labels = Variable(inputs), Variable(labels)
		outputs = net(inputs)
		loss_grad = criterion(outputs, labels)
		loss_grad.backward() # compute gradients
		full_loss_epoch[epoch] += loss_grad.item()

	#Compute epoch grad and epoch loss
	#Compute grad : see the 'clip_grad_norm' PyTorch function
	for para in net.parameters():
		grad_norm_epoch[epoch] += para.grad.data.norm(2)**2 #take the (norm2)**2 of parameter tensorFloat
	grad_norm_epoch[epoch] = (1./n_samples) * np.sqrt(grad_norm_epoch[epoch])
	full_loss_epoch[epoch] *= (1./n_samples)

	for i, data in enumerate(trainloader, 0):
		# get the inputs & wrap them in Variable
		inputs, labels = data
		inputs, labels = Variable(inputs), Variable(labels)
		# forward + backward + optimize
		net.zero_grad()
		outputs = net(inputs)
		loss = criterion(outputs, labels)
		loss.backward() # compute gradients
		for f in net.parameters():
			f.data.sub_(f.grad.data * learning_rate)

		# print statistics
		running_loss += loss.item()
		if i % 2500 == 2499:    # print every 2000 mini-batches
			print('[%d, %5d] loss: %.3f' %
					(epoch + 1, i + 1, running_loss / 2500))
			running_loss = 0.0

end = time.time()
print('time is : ', end - start)
print('Finished Training')

6250


TypeError: iter() returned non-iterator of type 'list'

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

outputs = net(Variable(images))
_, predicted = torch.max(outputs.data, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
epoch_abs = [i for i in range(n_epoch)]
plt.plot(epoch_abs, full_loss_epoch)
plt.xlabel('iteration')
plt.ylabel('value of objective function ')
plt.title('SGD : objective function evolution, log scale')
plt.yscale('log')
plt.grid()
plt.show()

plt.plot(epoch_abs, grad_norm_epoch, 'ro')
plt.xlabel('iteration')
plt.ylabel('gradient norm')
plt.title('SGD : grad norm evolution')
plt.grid()
plt.legend(loc=1)
plt.show()